In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import warnings
import time
import os.path
warnings.filterwarnings("ignore")

In [2]:
if not os.path.isfile("data/script/hillary_location.csv"):
    df = pd.read_csv("data/scraper/hillary.csv",sep=';')
    df = df.drop_duplicates(subset='id')
    df = df.drop(['retweets','favorites','replies','is_reply','is_retweet','url'], axis=1)
    df = df.dropna()

    df['user'] = df['user'].astype(str)
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
    df['id'] = df['id'].astype(int)
    df['text'] = df['text'].astype(str)
    df['userid'] = df['userid'].astype(int)

    df['loc'] = np.nan
    df['state'] = np.nan
    df['lat'] = np.nan
    df['lon'] = np.nan
    df['hashtag'] = np.nan

    df_location = pd.read_csv("data/script/hillary_users.csv")
    df_location = df_location[df_location['loc'].notnull()]
    df_location = df_location[df_location['lat'].notnull()]

In [3]:
def cleanText(s):
    s = s.split('http')[0]
    s = s.split('pic.twi')[0]
    h = [tag for tag in s.split() if tag.startswith("#") and len(tag) > 1]
    r = [tag for tag in s.split() if tag.startswith("@")]
    hr = h + r
    return ' '.join([t for t in s.split() if t not in hr]), h

if not os.path.isfile("data/script/hillary_location.csv"):
    df[['text','hashtag']] = df['text'].apply(lambda x: cleanText(x)).apply(pd.Series)

In [4]:
if not os.path.isfile("data/script/hillary_location.csv"):
    df['loc'] = df['userid'].map(df_location.set_index('userid')['loc'])
    df['lat'] = df['userid'].map(df_location.set_index('userid')['lat'])
    df['lon'] = df['userid'].map(df_location.set_index('userid')['lon'])

    df = df[df['loc'].notnull()]
    df = df[df['lat'].notnull()]
else:
    df = pd.read_csv("data/script/hillary_location.csv")
    print("There are %i unique tweets regarding %s that contain a location in the US" % (df.shape[0], "Hillary"))

In [5]:
def getState(coordinates):
    lat, lon, loc = coordinates
    states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    try:
        loc = [i for i in loc.replace(',',' ').split() if i in states]
        if len(loc) >= 1:
            return loc[-1]
        elif lat == 39.7837304 and lon == -100.4458825:
                return -1
        return np.nan
    except Exception as e:
        print(e)
        return np.nan


if not os.path.isfile("data/script/hillary_location.csv"):
    %time df['state'] = df[['lat','lon','loc']].apply(lambda coor: getState(coor),axis=1)
    df = df[df['state'] != -1]
    df = df.sort_values('state')
    df.to_csv("data/script/hillary_location.csv", index=False)

CPU times: user 12.4 s, sys: 133 ms, total: 12.5 s
Wall time: 12.9 s


In [6]:
number_states = df[df['state'].notnull()].shape[0]
print("You can start scraping for more states at", number_states)

You can start scraping for more states at 180364


In [7]:
import requests
import json

def getStateGoogle(coordinates):
    lat, lon, loc = coordinates
    url = "http://maps.googleapis.com/maps/api/geocode/json?"
    url += "latlng=%s,%s&sensor=false" % (lat, lon)
    #print(url)
    v = requests.get(url).text
    j = json.loads(v)
    components = j['results'][0]['address_components']
    time.sleep(1)
    for c in components:
        if "administrative_area_level_1" in c['types']:
            return c['short_name']
    return np.nan

try:
    for i in range(number_states,df.shape[0]):
        df['state'][i:i+1] = df[['lat','lon','loc']][i:i+1].apply(lambda coor: getStateGoogle(coor),axis=1)
except Exception as e:
    print(type(e),e)
    df.to_csv("data/script/hillary_location.csv", index=False)
        

<class 'IndexError'> ('list index out of range', 'occurred at index 14')


In [8]:
df

,user,date,id,text,userid,loc,state,lat,lon,hashtag
1459700,lizinthelibrary,2016-09-26 22:00:34,780587907216052224,She prepared for this debate and to be president.,54259621,"Anchorage, AK",AK,61.216313,-149.894852,"[#debatenight, #gethimgirl, #ImWithHer]"
201888,sharonmwinner,2016-11-05 05:39:04,794836420867129344,Who Broke Politics?,787819496048201728,"Anchorage, AK",AK,61.216313,-149.894852,[]
815484,LizjBradley,2016-10-19 21:44:51,788918872074166272,Taxes should be about investing in our future:...,425519723,"Napaskiak, AK",AK,60.708056,-161.766111,"[#ImWithHer, #debatenight]"
247795,keg2014gek,2016-11-03 20:45:02,794339637749628928,This Video Proves Hates Women -,2920257085,"Kodiak, AK",AK,57.790000,-152.407222,[]
811480,connie1776,2016-10-19 21:52:17,788920742222057472,what about all the campaign jobs she created?,278854821,"Anchorage, AK",AK,61.216313,-149.894852,"[#IMWITHHER, #debate]"
894779,DezariT,2016-10-17 13:13:05,788065306740600832,DONALD DRUMPF,458645640,"Anchorage, AK",AK,61.216313,-149.894852,"[#PLEASEGOTOTHELIGHT, #IDIOT, #WhenTheyGoLowWe..."
221736,keg2014gek,2016-11-04 16:40:08,794640393962782720,RealAlexJones Proven Right About Hillary Clint...,2920257085,"Kodiak, AK",AK,57.790000,-152.407222,"[#SpiritCooking, #Hillary2016]"
47362,sweetGM_Wms,2016-11-06 19:43:58,795426533481398272,"HillaryClinton no charges,I knew it. I'm telli...",747316888108077056,"Anchorage, AK",AK,61.216313,-149.894852,[#imwithher]
464636,angelaspeight,2016-10-26 01:07:21,791144162359926784,# ImWithHer,36883343,"Anchorage, AK",AK,61.216313,-149.894852,[]
269573,sharonmwinner,2016-11-03 05:55:57,794115891247992832,,787819496048201728,"Anchorage, AK",AK,61.216313,-149.894852,[]
